In [1]:
import csv
import json

### Load OpenImages Class Description

In [2]:
with open('data/openimages/class-descriptions-boxable.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    oi_classes = [row for row in csv_reader]

### Load WordNet Dump

In [3]:
# This WordNet dump was taken from https://github.com/nlp-compromise/wordnet.js/tree/master/data
with open('wordnet_js/Noun.js') as dataFile:
    data = dataFile.read()
    blub = data[data.find('{') : data.rfind('}')+1]
    
    # put keys in ""
    blub = blub.replace('id:', '"id":')
    blub = blub.replace('lexname:', '"lexname":')
    blub = blub.replace('syntactic_category:', '"syntactic_category":')
    blub = blub.replace('description:', '"description":')
    blub = blub.replace('words:', '"words":')
    blub = blub.replace('relationships:', '"relationships":')
    blub = blub.replace('type_of:', '"type_of":')
    blub = blub.replace('made_with:', '"made_with":')
    blub = blub.replace('members:', '"members":')
    blub = blub.replace('parts:', '"parts":')
    blub = blub.replace('instances:', '"instances":')
    blub = blub.replace('same_as:', '"same_as":')
    blub = blub.replace('freebase_topic:', '"freebase_topic":')
    blub = blub.replace('wikipedia_page:', '"wikipedia_page":')
    
    # turn into a list
    blub = '[' + blub + ']'
    
    # fix remaining errors
    blub = blub.replace('\'a complete extent or range: "a face that expressed a gamut of emotions"\'', '"a complete extent or range: a face that expressed a gamut of emotions"')
    blub = blub.replace('\'significant progress (especially in the phrase "make strides")\'', '"significant progress (especially in the phrase make strides)"')
    blub = blub.replace('\'the act of putting one thing or person in the place of another: "he sent Smith in for Jones but the substitution came too late to help"\'',
                        '"the act of putting one thing or person in the place of another: he sent Smith in for Jones but the substitution came too late to help"')
    blub = blub.replace('\'act of assembling and putting into readiness for war or other emergency: "mobilization of the troops"\'',
                        '"act of assembling and putting into readiness for war or other emergency: mobilization of the troops"')
    blub = blub.replace('\'a local branch where postal services are available"\'', '"a local branch where postal services are available"')
    blub = blub.replace('\'promise of reward as in "carrot and stick"\'', '"promise of reward as in carrot and stick"')
    blub = blub.replace('\'an area in which something acts or operates or has power or control: "the range of a supersonic jet"\'',
                        '"an area in which something acts or operates or has power or control: the range of a supersonic jet"')
    blub = blub.replace('\'a demand especially in the phrase "the call of duty"\'', '"a demand especially in the phrase the call of duty"')
    blub = blub.replace('\'the state of being kept down by unjust use of force or authority: "after years of oppression they finally revolted"\'',
                        '"the state of being kept down by unjust use of force or authority: after years of oppression they finally revolted"')
    blub = blub.replace('\'as the agent of or on someone\\\'s part (usually expressed as "on behalf of" rather than "in behalf of")\'',
                        '"as the agent of or on someone\'s part (usually expressed as on behalf of rather than in behalf of"')
    blub = blub.replace('\'a change in the units or form of an expression: "conversion from Fahrenheit to Centigrade"\'',
                        '"a change in the units or form of an expression: conversion from Fahrenheit to Centigrade"')
    blub = blub.replace('\'the quality of being able to meet a need satisfactorily: "he questioned the adequacy of the usual sentimental interpretation of the Golden Rule"\'',
                        '"the quality of being able to meet a need satisfactorily: he questioned the adequacy of the usual sentimental interpretation of the Golden Rule"')
    wordnet = json.loads(blub)

### Map Freebase Topics to WordNet IDs

In [4]:
fbtopic2wnid = {}
for synset in wordnet:
    if 'freebase_topic' in synset['same_as']:
        if synset['same_as']['freebase_topic'] in fbtopic2wnid:
            fbtopic2wnid[synset['same_as']['freebase_topic']].append(synset['id'])
        else:
            fbtopic2wnid[synset['same_as']['freebase_topic']] = [synset['id']]

### Count Matches

In [5]:
clean_match = 0
multi_match = 0

for cls in oi_classes:
    if cls[0] in fbtopic2wnid:
        if len(fbtopic2wnid[cls[0]]) == 1:
            clean_match += 1
        elif len(fbtopic2wnid[cls[0]]) > 1:
            multi_match += 1
            
print("clean matches: {}  multi matches: {}  no matches: {}".format(clean_match, multi_match, len(oi_classes) - multi_match - clean_match))

clean matches: 355  multi matches: 169  no matches: 77
